# Prompt-tuning

## step1 import lib

In [2]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer

## step2 load datasets

In [3]:
ds = Dataset.load_from_disk(r"../dataset/alpaca_data_zh")
ds

Dataset({
    features: ['output', 'input', 'instruction'],
    num_rows: 26858
})

In [4]:
ds[1:4]

{'output': ['4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。',
  '朱利叶斯·凯撒，又称尤利乌斯·恺撒（Julius Caesar）是古罗马的政治家、军事家和作家。他于公元前44年3月15日被刺杀。 \n\n根据历史记载，当时罗马元老院里一些参议员联合起来策划了对恺撒的刺杀行动，因为他们担心恺撒的统治将给罗马共和制带来威胁。在公元前44年3月15日（又称“3月的艾达之日”），恺撒去参加元老院会议时，被一群参议员包围并被攻击致死。据记载，他身中23刀，其中一刀最终致命。',
  '法国的首都是巴黎。'],
 'input': ['输入：4/16', '', ''],
 'instruction': ['解释为什么以下分数等同于1/4', '朱利叶斯·凯撒是如何死亡的？', '法国的首都是什么？']}

## step3 data process

In [5]:
model_path = r"D:\CodeLibrary\huggingface_model\Langboat\bloom-389m-zh"
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer

BloomTokenizerFast(name_or_path='D:\CodeLibrary\huggingface_model\Langboat\bloom-389m-zh', vocab_size=42437, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
def process_func(example):
    MAX_LENGTH = 256
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer("\n".join(["Human: "+ example["instruction"], example['input']]).strip() + "\n\nAssistant: ")
    response = tokenizer(example['output'] + tokenizer.eos_token)

    # 拼接
    input_ids = instruction['input_ids'] + response['input_ids']
    attention_mask = instruction['attention_mask'] + response['attention_mask']
    labels = [-100] * len(instruction['input_ids']) + response['input_ids']

    # 截断
    if len(input_ids) > MAX_LENGTH:
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }

In [7]:
tokenized_ds = ds.map(process_func, remove_columns=ds.column_names)
tokenized_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 26858
})

In [8]:
tokenizer.decode(tokenized_ds[1]['input_ids'])

'Human: 解释为什么以下分数等同于1/4\n输入：4/16\n\nAssistant: 4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

In [9]:
# decode看下 `labels`
tokenizer.decode(list(filter(lambda x : x != -100, tokenized_ds[1]['labels'])))

'4/16等于1/4是因为我们可以约分分子分母都除以他们的最大公约数4，得到（4÷4）/ (16÷4）=1/4。分数的约分是用分子和分母除以相同的非零整数，来表示分数的一个相同的值，这因为分数实际上表示了分子除以分母，所以即使两个数同时除以同一个非零整数，分数的值也不会改变。所以4/16 和1/4是两种不同的书写形式，但它们的值相等。</s>'

## step4 create model

In [10]:
model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

e:\software\anaconda3\envs\transformers\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
for param in model.parameters():
    print("param: ", param,  "\nnumbers: ", param.numel(), "\n", "="*100)

param:  Parameter containing:
tensor([[-0.0099, -0.0048, -0.0111,  ..., -0.0426,  0.0099,  0.0212],
        [ 0.0048, -0.0127,  0.0138,  ..., -0.0448,  0.0003, -0.0120],
        [ 0.0065,  0.0239,  0.0050,  ..., -0.0431, -0.0067,  0.0137],
        ...,
        [ 0.0331,  0.0185,  0.0101,  ..., -0.0432, -0.0090, -0.0024],
        [-0.0132,  0.0039,  0.0054,  ..., -0.0396, -0.0120,  0.0035],
        [ 0.0044,  0.0011,  0.0230,  ..., -0.0393, -0.0087, -0.0084]],
       requires_grad=True) 
numbers:  43455488 
param:  Parameter containing:
tensor([0.4409, 0.3167, 0.4749,  ..., 0.0816, 0.2927, 0.6006],
       requires_grad=True) 
numbers:  1024 
param:  Parameter containing:
tensor([-0.0513,  0.0164,  0.0052,  ...,  0.2412,  0.0072, -0.0292],
       requires_grad=True) 
numbers:  1024 
param:  Parameter containing:
tensor([0.6621, 0.8457, 0.5884,  ..., 3.2090, 0.9180, 0.5586],
       requires_grad=True) 
numbers:  1024 
param:  Parameter containing:
tensor([-0.0662, -0.0798,  0.0445,  ..., 

In [12]:
sum(param.numel() for param in model.parameters())

345768960

### 显存占用

model size: 0.35B

model: 0.35G * 4 ~= 1.4G

gradient: 0.35G * 4 ~= 1.4G

optimizer: 0.35G * 4 * 2 ~= 2.8G

sum: 5.6G

## Prompt-tuning 部分

### PEFT Step1 配置文件

`TaskType` 是指定微调任务 (本实验都是基于 `CasualLM` 的)

In [13]:
from peft import PromptTuningConfig, TaskType, get_peft_model, PromptTuningInit

# soft prompt; num_virtual_tokens是指定的 'prompt' token数量
# config = PromptTuningConfig(task_type=TaskType.CAUSAL_LM, num_virtual_tokens=10)
# config

# hard prompt
config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,   # 还有一个是RADNDOM，用了 TEXT 需要指定 init 的文本
    prompt_tuning_init_text="下面是一段人与机器人的对话。",
    num_virtual_tokens=len(tokenizer("下面是一段人与机器人的对话。")["input_ids"]),
    tokenizer_name_or_path=model_path,
)

config

PromptTuningConfig(peft_type=<PeftType.PROMPT_TUNING: 'PROMPT_TUNING'>, auto_mapping=None, base_model_name_or_path=None, revision=None, task_type=<TaskType.CAUSAL_LM: 'CAUSAL_LM'>, inference_mode=False, num_virtual_tokens=8, token_dim=None, num_transformer_submodules=None, num_attention_heads=None, num_layers=None, prompt_tuning_init=<PromptTuningInit.TEXT: 'TEXT'>, prompt_tuning_init_text='下面是一段人与机器人的对话。', tokenizer_name_or_path='D:\\CodeLibrary\\huggingface_model\\Langboat\\bloom-389m-zh', tokenizer_kwargs=None)

### PEFT Step2 创建模型

In [14]:
model = get_peft_model(model, config)
model

PeftModelForCausalLM(
  (base_model): BloomForCausalLM(
    (transformer): BloomModel(
      (word_embeddings): Embedding(42437, 1024)
      (word_embeddings_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (h): ModuleList(
        (0-23): 24 x BloomBlock(
          (input_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (self_attention): BloomAttention(
            (query_key_value): Linear(in_features=1024, out_features=3072, bias=True)
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (attention_dropout): Dropout(p=0.0, inplace=False)
          )
          (post_attention_layernorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): BloomMLP(
            (dense_h_to_4h): Linear(in_features=1024, out_features=4096, bias=True)
            (gelu_impl): BloomGelu()
            (dense_4h_to_h): Linear(in_features=4096, out_features=1024, bias=True)
          )
        )
      )

### soft prompt 模型信息
在上面的**模型信息**中，可以看到，通过 `get_peft_model` 将 `model` 打包后，原始的 `model` 套上了 `PeftModelForCausalLM`

在最后面我们可以看到下面内容:

```shell
(prompt_encoder): ModuleDict(
    (default): PromptEmbedding(
      (embedding): Embedding(10, 1024)
    )
  )
```

这个就是 `prompt` 部分，也就是需要微调部分的参数, 总共是 10*1024 = 10240 个参数

<br>
<br>

### transformers 本身提供查看 fine-tuned 参数的函数

可以看到是比之前 `bitfit` 还要少很多很多

<br>
<br>

### soft prompt 缺陷
参数太小了，基本上 `loss` 不会往下降; 要加大 epoch, 非常慢

In [15]:
# transformers 本身提供查看 fine-tuned 参数的函数

model.print_trainable_parameters()

trainable params: 8,192 || all params: 345,777,152 || trainable%: 0.0024


## step5 config train parameters

In [16]:
args = TrainingArguments(
    output_dir="./chatbot",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_steps=10,
    num_train_epochs=2
)

## step6 create trainer

In [17]:
trainer = Trainer(
    model=model,
    args=args,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
)

## step7 model train

In [18]:
trainer.train()

Step,Training Loss
10,3.141800
20,3.057400
30,3.026600
40,3.077400
50,3.118400
60,3.014200
70,3.057500
80,3.029100
90,2.978700
100,3.068900


TrainOutput(global_step=1678, training_loss=2.8501441589555525, metrics={'train_runtime': 1850.7385, 'train_samples_per_second': 29.024, 'train_steps_per_second': 0.907, 'total_flos': 1.164486099812352e+16, 'train_loss': 2.8501441589555525, 'epoch': 1.9991064780342516})

## 加载训练好的 PEFT 模型

In [21]:
from peft import PeftModel

model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)

peft_model = PeftModel.from_pretrained(model, "./chatbot/checkpoint-1500/")

## step8 model inference

### 方式一：
使用 `generate` 函数

In [25]:
peft_model = peft_model.cuda()
ipt = tokenizer("Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: ", return_tensors="pt").to(peft_model.device)
print(tokenizer.decode(peft_model.generate(**ipt, max_length=256, do_sample=True)[0], skip_special_tokens=True))

Human: 考试有哪些技巧？

Assistant: 考试技巧是指通过阅读、分析和运用知识技能，来提高考试技能的考试技巧，这些考试技巧包括，理解性阅读，听题，阅读练习与归纳，选择题，应用题，写作练习。


### 方式二：

使用 `pipeline`

In [26]:
from transformers import pipeline

pipe = pipeline("text-generation", model=peft_model, tokenizer=tokenizer, device=0)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'JambaForCausalLM', 'JetMoeForCausalLM', 'LlamaForCausalLM', 'MambaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalL

In [27]:
ipt = "Human: {}\n{}".format("考试有哪些技巧？", "").strip() + "\n\nAssistant: "
text = pipe(ipt, max_length=256, do_sample=True, )
text

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': 'Human: 考试有哪些技巧？\n\nAssistant: 考生可以从以下方面寻求帮助，如面试指导及帮助、心理测试、自我介绍、考官测评、写作指导等方面进行指导，这些都需要考生从自身努力方向出发。'}]